# Clase 4: Support Vector Machines

El objetivo de este ejercicio es consolidar la comprensión intuitiva de las SVM, y aprender experimentalmente a ajustar un modelo SVM.

*(Original adaptado del curso "Machine Learning" de Andrew NG, adaptado del material del curso "Aprendizaje Automático" de Cesar Olivares)*

## 1. SVM con kernel lineal

Empecemos con un conjunto de datos en 2D completamente separable por una frontera de decisión lineal. Importaremos los datos como un [`pandas.DataFrame`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html).

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
!wget 'https://drive.google.com/uc?export=download&id=19UGsjpICqXoTPqEsNUm7DDBlXkSQKtwl' -O ex6data1.csv
!wget 'https://drive.google.com/uc?export=download&id=1ISGGxpdb6YvrvWBCAzDzbUYN22y40oVg' -O ex6data2.csv
!wget 'https://drive.google.com/uc?export=download&id=1TTiu1cgekgb4b8YxsNGLM7NQAeUrdFaM' -O ex6data3val.csv
!wget 'https://drive.google.com/uc?export=download&id=1Jdz777tTPUf6ctCKe98WX0cjd4Rfsfjk' -O ex6data3train.csv

In [0]:
data1 = pd.read_csv('ex6data1.csv', names=['x1', 'x2', 'y'])

Siempre es bueno hacernos una idea inicial de los datos. Veamos sus principales parámetros estadísticos:

In [0]:
# Mostrar estadistica descriptiva
data1.describe()

Veamos ahora los primeros 5 del total de 51 elementos:

In [0]:
# Mostrar los primeros 5 elementos del conjunto de datos
data1.head()

También es posible mostrar fácilmente diagramas de caja:

In [0]:
data1.boxplot(return_type='axes')

Para mayor claridad, separaremos $X$ y $y$.

In [0]:
X = data1[['x1', 'x2']]
y = data1['y']

Veamos también la distribución de valores $y$:

In [0]:
print( 'Distribución de valores de y:' )
print( data1['y'].value_counts() )

data1['y'].value_counts().plot.bar()

Visualicemos $X$ en un diagrama de dispersión, usando colores para distinguir las etiquetas de clase $y \in [0, 1]$ con los colores rojo $(0)$ y verde $(1)$:

In [0]:
plt.title('Conjunto de datos 1')
plt.margins(x=0, y=0)
plt.xlabel('x1')
plt.ylabel('x2')
plt.scatter(X['x1'], X['x2'], c=y, s=30, cmap='prism');


En este conjunto de datos, la posición de los ejemplos sugiere una separación natural, un espacio o brecha entre ambos subconjuntos. Sin embargo, se puede observar también un valor atípico positivo en la parte superior izquierda $(0.1, 4.1)$. Como parte de este ejercicio vamos a ver cómo este valor atípico afecta la frontera de decisión de SVM.

### Efecto del parámetro $C$

En esta parte del ejercicio, vamos a probar ajustar modelos SVM con diferentes valores del parámetro $C$. Cuando $C$ es grande, el modelo SVM tratará de clasificar todos los ejemplos correctamente. $C$ juega un rol similar a $\frac{1}{\lambda}$, donde $\lambda$ es el parámetro de regularización estudiado en la regresión logística.

Vamos a usar SVC, que es una implementación del modelo SVM que viene incluida en scikit-learn y está basada en la librería *libsvm*.

Usemos en primer lugar un valor $C = 1$.

In [0]:
from sklearn.svm import SVC # "Support vector classifier"

# Ajuste del modelo SVM con C=1
model = SVC(kernel='linear', C=1)
model.fit(X, y)

Para visualizar mejor los datos, definiremos una función de apoyo.

In [0]:
# Tomado del libro 'Python Data Science Handbook' de Jake VanderPlas
def plot_svc_decision_function(model, ax=None, plot_support=True, levels=[-1, 0, 1], linestyles=['--', '-', '--']):
    """Plot the decision function for a 2D SVC"""
    if ax is None:
        ax = plt.gca()
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    # create grid to evaluate model
    x = np.linspace(xlim[0], xlim[1], 30)
    y = np.linspace(ylim[0], ylim[1], 30)
    Y, X = np.meshgrid(y, x)
    xy = np.vstack([X.ravel(), Y.ravel()]).T
    P = model.decision_function(xy).reshape(X.shape)
    
    # plot decision boundary and margins
    ax.contour(X, Y, P, colors='k',
               levels=levels, alpha=0.5,
               linestyles=linestyles)
    
    # plot support vectors
    if plot_support:
        ax.scatter(model.support_vectors_[:, 0],
                   model.support_vectors_[:, 1],
                   s=300, linewidth=1, facecolors='none');
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

Cuando $C = 1$, se observa que el modelo SVM coloca la frontera de decisión en la brecha que existe entre los dos subconjuntos de datos, y *clasifica mal* el valor atípico de la región superior izquierda. 

Los vectores de soporte están destacados con un círculo a su alrededor. Son todos aquellos que **no** se encuentran del lado correcto de la frontera de decisión con una distancia $m < 1$

In [0]:
# Visualización del modelo
plt.title('Frontera de decision SVM con C=1 (Conjunto de datos 1)')
plt.margins(x=0, y=0)
plt.xlabel('x1')
plt.ylabel('x2')
plt.scatter(X['x1'], X['x2'], c=y, s=30, cmap='prism');
plot_svc_decision_function(model)

A continuación, tu tarea es probar diferentes valores de $C$ en este conjunto de datos. Específicamente con los valores $C = 20$ y $C = 100$.

In [0]:
# Ajuste del modelo con C = 20
model20 = SVC(kernel='linear', C=20)
model20.fit(X, y)

# Visualización del modelo
plt.title('Frontera de decision SVM con C=20 (Conjunto de datos 1)')
plt.margins(x=0, y=0)
plt.xlabel('x1')
plt.ylabel('x2')
plt.scatter(X['x1'], X['x2'], c=y, s=30, cmap='prism');
plot_svc_decision_function(model20) 

In [0]:
# Ajuste del modelo con C = 100
model100 = SVC(kernel='linear', C=100)
model100.fit(X, y)

# Visualización del modelo
plt.title('Frontera de decision SVM con C=100 (Conjunto de datos 1)')
plt.margins(x=0, y=0)
plt.xlabel('x1')
plt.ylabel('x2')
plt.scatter(data1['x1'], data1['x2'], c=data1['y'], s=30, cmap='prism');
plot_svc_decision_function(model100) 

Contesta las siguientes preguntas en el cuestionario habilitado en la plataforma Paideia:

**Pregunta 1:** ¿Cuántos vectores de soporte resultan al entrenar el modelo SVM con un kernel lineal y un valor $C = 20$?

**Pregunta 2:** En el modelo con $C = 20$, ¿se clasifica correctamente el valor atípico ubicado en $(0.1, 4.1)$?

**Pregunta 3:** ¿Cuántos vectores de soporte resultan al entrenar el modelo SVM con un kernel lineal y un valor $C = 100$?

**Pregunta 4:** En el modelo con $C = 100$, ¿se clasifica correctamente el valor atípico ubicado en $(0.1, 4.1)$?

**Pregunta 5:** ¿La frontera de decisión del modelo $C = 100$ coincide con la separación natural entre los datos? 


## 2. SVM con kernel gaussiano

En esta parte del ejercicio usaremos SVMs para clasificación no lineal. Específicamente, usaremos SVMs con kernels gaussianos en conjuntos de datos que no son linealmente separables.

### 2.1. Conjunto de datos 2


In [0]:
data2 = pd.read_csv('ex6data2.csv', names=['x1', 'x2', 'y'])
X = data2[['x1', 'x2']]
y = data2['y']

data2.describe()

In [0]:
plt.title('Conjunto de datos 2')
plt.margins(x=0, y=0)
plt.xlabel('x1')
plt.ylabel('x2')
plt.scatter(X['x1'], X['x2'], c=y, s=10, cmap='prism');

Podemos observar que en el conjunto de datos 2 no hay una frontera de decisión lineal que pueda separar los ejemplos positivos y negativos. Sin embargo, usando SVM con kernel gaussiano, se puede aprender una frontera de decisión no lineal que se desempeñe razonablemente bien en este conjunto de datos.

Antes de usar un kernel gaussiano, es importante normalizar el conjunto de datos $X$. Si bien en este caso $x_1$ y $x_2$ ya tienen un mismo rango $[0, 1]$, aplicaremos de todos modos un [escalador estándar](http://scikit-learn.org/stable/modules/preprocessing.html#standardization-or-mean-removal-and-variance-scaling). El escalador estándar recibe $X$ y registra la media y desviación estándar, las cuales utilizará para transformar los datos. (Comparar con la estadística descriptiva dos pasos más arriba.)

In [0]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler().fit(X)
scaler

In [0]:
scaler.mean_

In [0]:
scaler.scale_

Para obtener los datos estandarizados, aplicamos la función `transform`:

In [0]:
X_scaled = scaler.transform(X)

`X_scaled` es un arreglo `numpy`. Usemos `pandas` para mostrar el sumario estadístico. Podemos verificar que los datos están normalizados con una media $\mu \approx 0$ y una desviación estándar $\sigma \approx 1$:

In [0]:
pd.DataFrame(X_scaled).describe()

Ahora sí ajustemos el modelo SVM con kernel gaussiano *(Radial Basis Function)*.

In [0]:
# Ajuste de modelo con kernel gaussiano
model = SVC(kernel='rbf', C=1, gamma=50)
model.fit(X_scaled, y)


Haremos la visualización con los datos normalizados. Dado que `X_scaled` es un arreglo `numpy`, accedemos a sus columnas $x_1$ y $x_2$ como `X_scaled[:,0]` y `X_scaled[:,1]`:

In [0]:
# Visualización
plt.title('Frontera de decision SVM con kernel gaussiano (Conjunto de datos 2 normalizado)')
plt.margins(x=0, y=0)
plt.xlabel('x1')
plt.ylabel('x2')
plt.scatter(X_scaled[:,0], X_scaled[:,1], c=y, s=10, cmap='prism');
plot_svc_decision_function(model, plot_support=False, levels=[0], linestyles=['-']) 

Por medio del método `score` podemos obtener la exactitud *(accuracy)* del modelo ajustado.

In [0]:
score = model.score(X_scaled, y)
tasa_de_error = (1- score)
n_ejemplos = len(X)
n_aciertos = score * len(X) 
n_errores = n_ejemplos - n_aciertos 

print ('Total de ejemplos: %d' % n_ejemplos)
print ('Exactitud: %0.4f' % score)
print ('Tasa de error: %0.4f' % tasa_de_error)
print ('Número de ejemplos correctamente clasificados: %d' % n_aciertos)
print ('Número de ejemplos incorrectamente clasificados: %d' % n_errores)


**Pregunta 6:** En el modelo ajustado al conjunto de datos 2, ¿cuántos ejemplos fueron correctamente clasificados?


### 2.2. Conjunto de datos 3

En esta parte del ejercicio vamos a aprender a seleccionar los parámetros $C$ y $\gamma$ requeridos por el kernel gaussiano.

La selección de los modelos de un parámetro se realiza por medio de *validación cruzada*. Para ello se reserva una muestra aleatoria del conjunto de datos que pueda servir para estimar el grado de generalización de los modelos a evaluar.

Para asegurar los mismos resultados, en este ejercicio recibimos directamente el conjunto de entrenamiento `(X, y)` y el conjunto de validación `(Xval, yval)`.

In [0]:
data3train = pd.read_csv('ex6data3train.csv', names=['x1', 'x2', 'y'])
X = data3train[['x1', 'x2']]
y = data3train['y']

data3val = pd.read_csv('ex6data3val.csv', names=['x1', 'x2', 'y'])
Xval = data3val[['x1', 'x2']]
yval = data3val['y']

plt.title('Conjunto de datos 3')
plt.margins(x=0, y=0)
plt.xlabel('x1')
plt.ylabel('x2')
plt.scatter(X['x1'], X['x2'], c=y, s=10, cmap='prism', label='Conjunto de entrenamiento')
plt.scatter(Xval['x1'], Xval['x2'], c=yval, marker='x', s=20, cmap='prism', label='Conjunto de validacion')
plt.show()


Realizaremos también la normalización de los datos. Presta atención a que la normalización de los datos de validación $X_{val}$ se debe realizar con los parámetros obtenidos del conjunto de entrenamiento $X$.

In [0]:
scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.transform(X)
Xval_scaled = scaler.transform(Xval)

Comparemos la exactitud en los conjuntos de entrenamiento y validación para un modelo con $C=200$ y $\gamma = 50$

In [0]:
model = SVC(kernel='rbf', C=200, gamma=50)
model.fit(X_scaled, y)

score_train = model.score(X_scaled, y)
score_val = model.score(Xval_scaled, yval)

print ('Exactitud en el conjunto de entrenamiento: %0.4f' % score_train)
print ('Exactitud en el conjunto de validación: %0.4f' % score_val)


Si bien se tiene una exactitud muy alta en el conjunto de entrenamiento, la exactitud es bastante menor en el conjunto de validación. Este es un síntoma claro de sobreajuste *(overfitting)*. Visualicemos lo que está ocurriendo.

In [0]:
# Visualización
plt.title('SVM con kernel gaussiano (C=200, gamma=50) (Conjunto de datos 3 normalizado)')
plt.margins(x=0, y=0)
plt.xlabel('x1')
plt.ylabel('x2')
plt.scatter(X_scaled[:,0], X_scaled[:,1], c=y, s=10, cmap='prism', label='Conjunto de entrenamiento')
plt.scatter(Xval_scaled[:,0], Xval_scaled[:,1], c=yval, marker='x', s=20, cmap='prism', label='Conjunto de validacion')
plot_svc_decision_function(model, plot_support=False, levels=[0], linestyles=['-']) 


Podemos apreciar visualmente que la generalización es deficiente porque el modelo está *"memorizando"* la ubicación de los valores atípicos. Este no es un comportamiento deseable.

Para determinar valores $C$ y $\gamma$ que brinden una mejor generalización, es necesario probar con diferentes valores, ajustando el modelo en el conjunto de entrenamiento y probando su desempeño en el conjunto de validación. Examina y completa el siguiente código:

In [0]:
Cs = np.logspace(-2,2,9)  # ~ [0.01, 0.03, 0.1, ..., 100]
gammas = np.logspace(-4,4,9)  # [0.0001, 0.001, ..., 10000]

mejor_modelo = None
mejor_score = 0
for C in Cs:
    for gamma in gammas:
        
        model = SVC(kernel='rbf', C=C, gamma=gamma) 
        model.fit(X_scaled, y)
        
        score_val = model.score(Xval_scaled, yval) 
        
        if score_val > mejor_score:
            mejor_score = score_val
            mejor_modelo = model


print ('Mejor valor de C: %0.4f' % mejor_modelo.get_params()['C'])
print ('Mejor valor de gamma: %0.4f' % mejor_modelo.get_params()['gamma'])
print ('Exactitud en el conjunto de entrenamiento: %0.4f' % mejor_modelo.score(X_scaled, y))
print ('Exactitud en el conjunto de validación: %0.4f' % mejor_modelo.score(Xval_scaled, yval))

# Visualización
plt.title('SVM con kernel gaussiano (Conjunto de datos 3)')
plt.margins(x=0, y=0)
plt.scatter(X_scaled[:,0], X_scaled[:,1], c=y, s=10, cmap='prism', label='Conjunto de entrenamiento')
plt.scatter(Xval_scaled[:,0], Xval_scaled[:,1], c=yval, marker='x', s=20, cmap='prism', label='Conjunto de validacion')
plot_svc_decision_function(mejor_modelo, plot_support=False, levels=[0], linestyles=['-']) 


Observa la forma de la frontera de decisión de tu mejor modelo y compárala con la obtenida arriba con $C=200$ y $\gamma = 50$. Luego responde las siguientes preguntas sobre tu mejor modelo:

**Pregunta 7:** ¿Cuál fue el mejor valor de $C$ obtenido?

**Pregunta 8:** ¿Cuál fue el mejor valor de $\gamma$ obtenido?

**Pregunta 9:** ¿Se obtuvo una mejor exactitud en el conjunto de entrenamiento o en el conjunto de validación?

# Aplicaciones prácticas: Breast Cancer

Carga de datos y separación en conjuntos de entrenamiento y validación.

In [0]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler

data = load_breast_cancer()
Xtrain, Xtest, Ytrain, Ytest = train_test_split(data.data, data.target, test_size=0.33)

Preprocesamiento de datos

In [0]:
scaler = StandardScaler()
Xtrain = scaler.fit_transform(Xtrain)
Xtest = scaler.transform(Xtest)

Entrenamiento del modelo y obtención del score.

In [0]:
model = SVC(kernel='rbf',C=4., gamma=.05)
model.fit(Xtrain, Ytrain)
print("train score:", model.score(Xtrain, Ytrain))
print("test score:", model.score(Xtest, Ytest))

Exploración de otras métricas:

In [0]:
from sklearn.metrics import classification_report
Ypred = model.predict(Xtest)

In [0]:
print(classification_report(Ytest,Ypred))

Pruebas con kernel linear

In [0]:
model = SVC(kernel='linear',C=4.)
model.fit(Xtrain, Ytrain)
print("train score:", model.score(Xtrain, Ytrain))
print("test score:", model.score(Xtest, Ytest))

Pruebas con kernel polinomial de grados 2, 3 y 4

In [0]:
model = SVC(kernel='poly',C=4.,degree=2)
model.fit(Xtrain, Ytrain)
print("train score:", model.score(Xtrain, Ytrain))
print("test score:", model.score(Xtest, Ytest))

In [0]:
model = SVC(kernel='poly',C=4.,degree=3)
model.fit(Xtrain, Ytrain)
print("train score:", model.score(Xtrain, Ytrain))
print("test score:", model.score(Xtest, Ytest))

In [0]:
model = SVC(kernel='poly',C=4.,degree=4)
model.fit(Xtrain, Ytrain)
print("train score:", model.score(Xtrain, Ytrain))
print("test score:", model.score(Xtest, Ytest))

Pruebas con kernel Sigmoidal

In [0]:
model = SVC(kernel='sigmoid',C=4.,gamma=.05)
model.fit(Xtrain, Ytrain)
print("train score:", model.score(Xtrain, Ytrain))
print("test score:", model.score(Xtest, Ytest))

In [0]:
Cs = np.logspace(-2,2,9)  # ~ [0.01, 0.03, 0.1, ..., 100]
gammas = np.logspace(-4,4,9)  # [0.0001, 0.001, ..., 10000]

mejor_modelo = None
mejor_score = 0
for C in Cs:
    for gamma in gammas:
        
        model = SVC(kernel='rbf', C=C, gamma=gamma) 
        model.fit(Xtrain, Ytrain)
        
        score_val = model.score(Xtest, Ytest) 
        
        if score_val > mejor_score:
            mejor_score = score_val
            mejor_modelo = model


print ('Mejor valor de C: %0.4f' % mejor_modelo.get_params()['C'])
print ('Mejor valor de gamma: %0.4f' % mejor_modelo.get_params()['gamma'])
print ('Exactitud en el conjunto de entrenamiento: %0.8f' % mejor_modelo.score(Xtrain, Ytrain))
print ('Exactitud en el conjunto de validación: %0.8f' % mejor_modelo.score(Xtest, Ytest))